In [ ]:
!pip install transformers
!pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 34.5 MB/s 
     |████████████████████████████████| 596 kB 70.4 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 6.6 MB 34.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 43.0 MB/s 
     |████████████████████████████████| 3.5 MB 60.8 MB/s 
     |████████████████████████████████| 527 kB 69.9 MB/s 
     |████████████████████████████████| 141 kB 72.4 MB/s 
     |████████████████████████████████| 829 kB 64.8 MB/s 
     |████████████████████████████████| 419 kB 69.1 MB/s 
     |████████████████████████████████| 952 kB 54.2 MB/s 
     |█████████

In [ ]:
import pandas as pd
from simplet5 import SimpleT5
from sklearn.model_selection import train_test_split

Global seed set to 42


In [ ]:
df = pd.read_csv("/content/MyDrive/MyDrive/final.csv")

In [ ]:
df

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,2,A-63849,But what's the matter?,What's wrong?
3,3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."
...,...,...,...,...
51782,205,40471-1502742711264-62106,He hath not told us of the captain yet.,He hasn't told us about that captain yet.
51783,206,40471-1502742725153-49869,"When that is known and golden time convents, ...",When that's taken care of and the time is conv...
51784,207,40471-1502742735040-34786,"Meantime, sweet sister, We will not part fro...","Until then, sweet sister-in-law, we won't leav..."
51785,208,40471-1502742744235-12820,"Cesario, come, For so you shall be, while yo...","Cesario, come here. You'll be Cesario to me wh..."


In [ ]:
subset = df[df.columns[2:]].copy()

In [ ]:
subset.loc[:,"t"] =subset.loc[:,"t"].str.strip()
subset.loc[:,"og"] =subset.loc[:,"og"].str.strip()

In [ ]:
subset.rename(columns={"og":"target_text","t":"source_text"},inplace=True)
# subset = subset[subset['source_text','target_text']]

In [ ]:
subset["source_text"] = "translate: "+subset["source_text"]

In [ ]:
train_df, test_df = train_test_split(subset, test_size=0.2)
train_df.shape, test_df.shape

((41429, 2), (10358, 2))

In [ ]:
model = SimpleT5()
# model.from_pretrained("")
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:10000],
            eval_df=test_df[:500], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=6, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
mod = transformers.AutoModelForSeq2SeqLM.from_pretrained("/content/outputs/simplet5-epoch-5-train-loss-1.6847-val-loss-2.3567")

In [ ]:
tokenizer= transformers.T5Tokenizer.from_pretrained("t5-base")

In [ ]:
f = lambda x: tokenizer.decode(mod.generate(tokenizer(x,return_tensors="pt").input_ids)[0])

In [ ]:
k = 72 #@param {type:"slider", min:0, max:400, step:1}

s = test_df.iloc[k]["source_text"]
t = test_df.iloc[k]["target_text"]
print(f"Source: {s}")
print(f"Target: {t}")
print(f"Model: {f(s)}")

Source: translate: It felt like it was a brother daring another brother to practice fighting with him.
Target: Unless a brother should a brother dare  To gentle exercise and proof of arms.
Model: <pad> It was as if a brother dares another brother to fight with him.</s>


In [ ]:
print(f("translate: This name is nice."))

<pad> This is a fair name.</s>


In [ ]:
zip -r outputs.zip /content/outputs

	zip warning: name not matched: /content/MyDrive/MyDrive/outputs.zip

zip error: Nothing to do! (try: zip -r /content/outputs.zip . -i /content/MyDrive/MyDrive/outputs.zip)
